# Environment

In [1]:
import higlass
from higlass.client import View, Track, CombinedTrack
from higlass.tilesets import cooler, beddb, chromsizes, bigwig, Tileset
from higlass.viewer import save_b64_image_to_png
import clodius
import os
import os.path as path
import pandas as pd
import numpy as np
import itertools
import negspy.coordinates as nc

def bed2ddb(filepath, uuid=None, **kwargs):
    from clodius.tiles.bed2ddb import get_2d_tileset_info, get_2D_tiles
    from clodius.tiles.utils import tiles_wrapper_2d

    return Tileset(
        uuid=uuid,
        tileset_info=lambda: get_2d_tileset_info(filepath),
        tiles=lambda tids: tiles_wrapper_2d(
            tids, lambda z, x, y: get_2D_tiles(filepath, z, x, y)[(x, y)]
        ),
        **kwargs
    )


# Annotations

In [76]:
annot_dir = path.join("..", "Results", "2019-10-24_higlass")
genes = beddb(path.join(annot_dir, "Data", "hg38", "gene-annotations-hg38.beddb"))
chrom_sizes = chromsizes(path.join(annot_dir, "hg38.chrom.sizes"))
label_font_size = 18
chr_label_size = 30
annots_size = 150

chrom_labels = {
    p: Track(
        track_type=l + "-chromosome-labels",
        tileset=chrom_sizes,
        position=p,
        height=chr_label_size,
        width=chr_label_size,
        options={
            "fontSize": label_font_size,
            "showMousePosition": True,
        },
    ) for p, l in zip(["top", "bottom", "left", "right"], ["horizontal", "horizontal", "vertical", "vertical"])
}
chrom_grid = {
    p: Track(
        track_type=l + "-chromosome-grid",
        tileset=chrom_sizes,
        position=p,
        height=chr_label_size,
        width=chr_label_size,
        options={
            "fontSize": label_font_size,
            "showMousePosition": True,
        },
    ) for p, l in zip(
        ["top", "bottom", "left", "right"],
        ["horizontal", "horizontal", "vertical", "vertical"])
}

gene_annots = {
    p: Track(
        track_type=l + "-gene-annotations",
        tileset=genes,
        position=p,
        height=annots_size,
        width=annots_size,
        options={
            "fontSize": label_font_size,
            "showMousePosition": True,
        },
    ) for p, l in zip(["top", "bottom", "left", "right"], ["horizontal", "horizontal", "vertical", "vertical"])
}

hg38 = nc.get_chrominfo("hg38")

# Load data

## Metadata

In [61]:
meta = pd.read_csv("config.tsv", sep="\t")
meta = meta.loc[meta.Include == "Yes", :]

SAMPLES = {
    "all": meta["Sample_ID"].tolist(),
    "tumour": meta.loc[(meta.Source == "Primary") & (meta.Type == "Malignant"), "Sample_ID"].tolist(),
    "benign": meta.loc[(meta.Source == "Primary") & (meta.Type == "Benign"), "Sample_ID"].tolist(),
    "primary": meta.loc[(meta.Source == "Primary"), "Sample_ID"].tolist(),
    "clines": meta.loc[(meta.Source == "Cell Line"), "Sample_ID"].tolist(),
}

## Contact Matrices

In [129]:
resolutions = [
    1000, 2000, 3000, 4000, 5000,
    10000, 20000, 30000, 40000, 50000,
    100000, 200000, 300000, 400000, 500000,
    1000000, 2000000, 3000000, 4000000, 5000000
][::-1]
min_resolution = 40000
colour_range = [
    "rgba(65, 105, 225, 1.0)",
    "rgba(255, 250, 250, 1.0)",
    "rgba(255, 25, 25, 1.0)"
]

mtx_dir = {
    "primary": path.join("..", "Data", "Processed", "2019-06-18_PCa-LowC-sequencing", "Contacts"),
    "clines": path.join("..", "..", "Data", "External", "Rhie_2019", "Contacts"),
}
mtx_files = {}
for s in SAMPLES["primary"]:
    mtx_files[s] = path.join(mtx_dir["primary"], s + ".mcool")

for s in SAMPLES["clines"]:
    mtx_files[s] = path.join(mtx_dir["clines"], s + ".mcool")

mtx_tilesets = {
    s: cooler(f)
    for s, f in mtx_files.items()
}

contacts = {
    s: {
        p: Track(
            track_type=pl + "heatmap",
            datatype="matrix",
            filetype="cooler",
            tileset=mtx_tilesets[s],
            height=height,
            options={
                "maxZoom": str(resolutions.index(min_resolution)),
                "colorbarPosition": "topRight",
                "showMousePosition": True,
                "name": meta.loc[meta["Sample_ID"] == s, "Label"].values[0],
                "colorRange": colour_range,
            }
        )
        for p, pl, height in zip(
            ["top", "right", "bottom", "left", "center"],
            ["horizontal-", "horizontal-", "horizontal-", "horizontal-", ""],
            #["linear-", "linear-", "linear-", "linear-", ""],
            [120, 120, 120, 120, 240],
        )
    }
    for s in mtx_files.keys()
}

In [130]:
views = [View(
    tracks=[
        chrom_labels["top"],
        gene_annots["top"],
    ] + [
        contacts[s]["top"] for s in SAMPLES["primary"]
    ],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr1", 0, hg38),
        nc.chr_pos_to_genome_pos("chrM", 0, hg38)
    ],
)]

d, s, v = higlass.display(
    views=views,
    zoom_syncs=[views],
    location_syncs=[views],
)
d

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'fWR8HyIzSWSndbfE4OFBAg', 'tracks': {'top': [{'ty…

## TADs

In [4]:
windows = list(range(3, 21))
tad_track_dir = path.join("..", "Results", "2020-08-29_TADs-downsampled", "Tracks")

In [117]:
tad_files = {
    "1D": {
        s: {
            w: path.join(tad_track_dir, s + ".120000000.res_40000bp.domains.window_" + str(w) + ".beddb")
            for w in windows
        }
        for s in SAMPLES["all"]
    },
    "2D": {
        s: path.join(tad_track_dir, s + ".120000000.res_40000bp.domains.all.bed2ddb")
        for s in SAMPLES["all"]
    },
}
tad_tilesets = {
    "1D": {
        s: {
            w: beddb(
                tad_files["1D"][s][w],
                name=meta.loc[meta.Sample_ID == s, "Label"].values[0] + "TADs (w = " + str(w) +")"
            )
            for w in windows
        }
        for s in SAMPLES["all"]
    },
    "2D": {
        s: bed2ddb(tad_files["2D"][s])
        for s in SAMPLES["all"]
    }
}
tads = {
    "1D": {
         s: {
             w: {
                p: Track(
                    track_type="bedlike",
                    datatype="bedlike",
                    filetype="beddb",
                    position=p,
                    tileset=tad_tilesets["1D"][s][w],
                    options={
                        "showMousePosition": True,
                        "name": meta.loc[meta.Sample_ID == s, "Label"].values[0],
                        "labelPosition": "topLeft",
                        "fillColor": meta.loc[meta.Sample_ID == s, "Sample_Colour"].values[0],
                        "labelColor": meta.loc[meta.Sample_ID == s, "Sample_Colour"].values[0]
                    },
                )
                for p in ["top", "right", "bottom", "left"]
            }
            for w in windows
        }
        for s in SAMPLES["all"]   
    },
    "2D": {
         s: {
            p: Track(
                track_type=pl+"2d-rectangle-domains",
                datatype="2d-rectangle-domains",
                filetype="bed2ddb",
                position=p,
                tileset=tad_tilesets["2D"][s],
                height=50,
                width=50,
                options={
                    "showMousePosition": True,
                    "name": meta.loc[meta.Sample_ID == s, "Label"].values[0],
                    "labelPosition": "topLeft",
                    "labelColor": meta.loc[meta.Sample_ID == s, "Sample_Colour"].values[0],
                    "rectangleDomainFillColor": meta.loc[meta.Sample_ID == s, "Sample_Colour"].values[0],
                    "rectangleDomainStrokeColor": meta.loc[meta.Sample_ID == s, "Sample_Colour"].values[0],
                    "rectangleDomainOpacity": 0.01,
                },
            )
            for p, pl in zip(
                ["top", "right", "bottom", "left", "center"],
                # later versions of HiGlass switch "horizontal"/"vertical" to "linear"
                ["horizontal-", "vertical-", "horizontal-", "vertical-", ""],
            )
        }
        for s in SAMPLES["all"]   
    },
}

# a test track to show that 2D tracks are being displayed
#ts = bed2ddb(path.join(tad_track_dir, "tmp.bedpedb"))
#(d,s,v) = higlass.display([
#    View([
#        Track('top-axis'),
#        Track('2d-rectangle-domains', position='center', tileset=ts, height=400),
#    ], initialXDomain=[0, 20e6])
#])
#d

In [118]:
views = [View(
    tracks=[
        chrom_labels["top"],
        gene_annots["top"],
    ] + [
        tads["2D"][s]["top"] for s in SAMPLES["primary"]
    ],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr1", 0, hg38),
        nc.chr_pos_to_genome_pos("chrM", 0, hg38)
    ],
)]

d, s, v = higlass.display(
    views=views,
    zoom_syncs=[views],
    location_syncs=[views],
)
d

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'KBuIqw_0RmChiXkyYgzGww', 'tracks': {'top': [{'ty…

## Compartments

In [62]:
cmpt_files = {
    s: path.join("..", "Results", "2021-03-03_compartments", "Compartments", s + ".compartments.cis.bw")
    for s in SAMPLES["all"]
}
cmpt_tilesets = {s: bigwig(f) for s, f in cmpt_files.items()}
compartments = {
    s: {
        p: Track(
            track_type=t + "-line",
            filetype="vector",
            position=p,
            tileset=ts,
            height=100,
            options={
                "showMousePosition": True,
                "name": meta.loc[meta["Sample_ID"] == s, "Label"].values[0] + " Compartment",
                "lineStrokeColor": meta.loc[meta["Sample_ID"] == s, "Sample_Colour"].values[0],
                "valueScaleMin": -2,
                "valueScaleMax": 2,
                "zeroLineVisible": True,
                "zeroLineColor": "#000000",
            },
        ) for p, t in zip(
            ["top", "bottom", "left", "right"],
            ["horizontal", "horizontal", "vertical", "vertical"]
        )
    } for s, ts in cmpt_tilesets.items()
}

In [66]:
views = [View(
    tracks=[
        chrom_labels["top"],
        gene_annots["top"],
    ] + [
        compartments[s]["top"] for s in SAMPLES["primary"]
    ],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr1", 0, hg38),
        nc.chr_pos_to_genome_pos("chrM", 0, hg38)
    ],
)]

d, s, v = higlass.display(
    views=views,
    zoom_syncs=[views],
    location_syncs=[views],
    value_scale_syncs=[
        [(views[0], compartments[s]["top"]) for s in SAMPLES["primary"]],
    ],
)
d

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'ALQZ_u8YQmKjZSbsxbFceQ', 'tracks': {'top': [{'ty…

### Structural variants

In [82]:
sv_dir = path.join("..", "Results", "2020-02-19_chromoplexy")
sv_track_dir = path.join(sv_dir, "Tracks")
breaks = pd.read_csv(
    path.join(sv_dir, "Graphs", "sv-breakpoints.paired.tsv"),
    sep="\t",
    header=0,
    index_col=False,
)

sv_files = {
    "1D": {
        s: path.join(sv_track_dir, s + ".breaks.beddb")
        for s in SAMPLES["tumour"]
    },
    "2D": {
        s: path.join(sv_track_dir, s + ".breaks.bed2ddb")
        for s in SAMPLES["tumour"]
    }
}
sv_tilesets = {
    "1D": {
        s: beddb(
            v,
            name=meta.loc[meta["Sample_ID"] == s, "Label"].values[0] + " Breaks"
        )
        for s, v in sv_files["1D"].items()
    },
    "2D": {
        s: bed2ddb(
            v,
            name=meta.loc[meta["Sample_ID"] == s, "Label"].values[0] + " Breaks"
        )
        for s, v in sv_files["2D"].items()
    }
}
svs = {
    "1D": {
        s: {
            p: Track(
                track_type=t + "bedlike",
                datatype="bedlike",
                filetype="beddb",
                position=p,
                tileset=sv_tilesets["1D"][s],
                options={
                    "showMousePosition": True,
                    "name": meta.loc[meta["Sample_ID"] == s, "Label"].values[0],
                    "labelPosition": "topLeft",
                    "labelColor": meta.loc[meta.Sample_ID == s, "Sample_Colour"].values[0],
                    "fillColor": meta.loc[meta.Sample_ID == s, "Sample_Colour"].values[0],
                },
            ) for p, t in zip(
                ["top", "bottom", "left", "right"],
                ["", "", "vertical-", "vertical-"])
        }
        for s in SAMPLES["tumour"]
    },
    "2D": {
        s: {
            p: Track(
                track_type=pl+"2d-rectangle-domains",
                datatype="2d-rectangle-domains",
                filetype="bed2ddb",
                position=p,
                tileset=sv_tilesets["2D"][s],
                height=50,
                width=50,
                options={
                    "showMousePosition": True,
                    "name": meta.loc[meta.Sample_ID == s, "Label"].values[0],
                    "labelPosition": "topLeft",
                    "labelColor": meta.loc[meta.Sample_ID == s, "Sample_Colour"].values[0],
                    "rectangleDomainFillColor": meta.loc[meta.Sample_ID == s, "Sample_Colour"].values[0],
                },
            )
            for p, pl in zip(
                ["top", "right", "bottom", "left", "center"],
                # later versions of HiGlass switch "horizontal"/"vertical" to "linear"
                ["horizontal-", "vertical-", "horizontal-", "vertical-", ""],
            )
        }
        for s in SAMPLES["tumour"]
    }
}

In [68]:
views = [View(
    tracks=[
        chrom_labels["top"],
        gene_annots["top"],
    ] + [
        svs["1D"][s]["top"] for s in SAMPLES["tumour"]
    ],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr1", 0, hg38),
        nc.chr_pos_to_genome_pos("chrM", 0, hg38)
    ],
)]

d, s, v = higlass.display(
    views=views,
    server_port=8193,
    zoom_syncs=[views],
    location_syncs=[views],
)
d

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'cICuDo-ZQ3msI4yQ55-JpQ', 'tracks': {'top': [{'ty…

### Copy Number

In [74]:
cnv_track_dir = path.join("..", "Data", "Processed", "2019-06-18_PCa-LowC-sequencing", "Tracks")
cnv_files = {
    s: path.join(cnv_track_dir, s + ".cnv.bw")
    for s in SAMPLES["all"]
}
cnv_tilesets = {
    s: bigwig(f)
    for s, f in cnv_files.items()
}
cnvs = {
    s: {
        p: Track(
            track_type=t + "-line",
            datatype="vector",
            filetype="vector",
            position=p,
            tileset=ts,
            height=100,
            options={
                "showMousePosition": True,
                "name": meta.loc[meta["Sample_ID"] == s, "Label"].values[0] + " log2CopyRatio",
                "lineStrokeColor": meta.loc[meta["Sample_ID"] == s, "Sample_Colour"].values[0],
                "valueScaleMin": -2,
                "valueScaleMax": 2,
            },
        ) for p, t in zip(["top", "bottom", "left", "right"], ["horizontal", "horizontal", "vertical", "vertical"])
    } for s, ts in cnv_tilesets.items()
}

In [81]:
views = [View(
    tracks=[
        chrom_labels["top"],
        chrom_grid["top"],
        gene_annots["top"],
    ] + [
        cnvs[s]["top"] for s in SAMPLES["primary"]
    ],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr1", 0, hg38),
        nc.chr_pos_to_genome_pos("chrM", 0, hg38)
    ],
)]

d, s, v = higlass.display(
    views=views,
    server_port=8193,
    zoom_syncs=[views],
    location_syncs=[views],
    value_scale_syncs=[
        [(views[0], cnvs[s]["top"]) for s in SAMPLES["primary"]],
    ],
)
d

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'CuH2dZ63Qx2awBosfmpDsA', 'tracks': {'top': [{'ty…